## UCA Rule Engine Notebook

The **UCA Rule Engine** lets you define and manage rules that guide data and model workflows.  
A "rule" is essentially an instruction that connects your data (like folders, counts, or sample images) with services (such as data labeling or model training).  

This notebook helps you try the Rule Engine end-to-end - from creating a rule to listing, updating, and deleting it.  

Just run the cells in order to see how it works.

### 1) Setup

Installing dependencies

In [36]:
!pip -q install requests tapipy python-dotenv

Importing packages

In [37]:
from getpass import getpass
from pprint import pprint

import requests
from tapipy.tapis import Tapis

### 2) Configure API

**Side Note:**  
As of now, the `BASE_URL` will vary depending on where the Rule Engine is hosted.  

- For local runs, you can use ngrok (or a similar tool) to generate a temporary URL as Notebook doesn't support `localhost`
- In dev or shared environments, this will be updated to the appropriate domain once we learn more about the existing ICICLE practices for hosting servers.    

Please update the `BASE_URL` in this notebook to match the environment you are testing against.

In [38]:
# Server URL
BASE_URL = "https://df42ffe69d9d.ngrok-free.app" # Example URL
print("BASE_URL =", BASE_URL)

BASE_URL = https://df42ffe69d9d.ngrok-free.app


### 3) Tapis authentication

- Enter Tapis tenant, username, and password in the cell.
- The notebook fetches an access token.

In [40]:
# Tapis login (client-side only)
TAPIS_URL = "https://tacc.tapis.io"   # or your tenant base
USERNAME  = input("username : ")
PASSWORD  = getpass("password : ")

t = Tapis(base_url=TAPIS_URL, username=USERNAME, password=PASSWORD)
t.get_tokens()
ACCESS_TOKEN = t.access_token.access_token
print("Access token acquired (length):", len(ACCESS_TOKEN))

username : molakalmuru
password : ··········
Access token acquired (length): 912


#### Setting up Authentication Helpers

Here we define small helpers to make authenticated requests:

- **Headers**: Attaching Tapis access token (`Authorization: Bearer ...`).
- **Params**: Optionally send the token as a query param (`?tapis_token=...`).
- **Raise with body**: Collects error details so we can see why a call failed.

we won’t need to touch these once set up — the rest of the notebook will use them automatically

In [41]:
S = requests.Session()
# Preferred: Authorization header
HEADERS = {"Authorization": f"Bearer {ACCESS_TOKEN}", "Content-Type": "application/json"}

# Optional: if you enabled query param auth in the server (`tapis_token`)
USE_QUERY_TOKEN = False  # set True only if you want to send ?tapis_token=<token>

def _headers():
    return None if USE_QUERY_TOKEN else HEADERS

def _params(extra: dict | None = None):
    p = {}
    if USE_QUERY_TOKEN:
        p["tapis_token"] = ACCESS_TOKEN
    if extra:
        p.update(extra)
    return p or None

def _raise_with_body(resp: requests.Response):
    try:
        body = resp.json()
    except Exception:
        body = resp.text
    status = resp.status_code
    msg = f"HTTP {status}: {body}"
    resp.raise_for_status()
    raise RuntimeError(msg)


### 4) Health Check

- Quick ping to `/health`.
- Expect: `{ "status": "ok" }`

In [42]:
def health():
    r = S.get(f"{BASE_URL}/health", headers=_headers(), params=_params(), timeout=20)
    if r.status_code >= 400: 
        _raise_with_body(r)
    return r.json()

print("Health:", health())

Health: {'status': 'ok'}



### 5) Create a Rule

This step adds a new rule for the user into the Rule Engine.  
When you create a rule, it gets stored in the database and linked to your user identity.  


In [43]:
def create_rule(rule: dict) -> str:
    r = S.post(f"{BASE_URL}/rules", headers=_headers(), params=_params(),
               json=rule, timeout=30)
    if r.status_code >= 400: 
        _raise_with_body(r)
    return r.json()["Rule_UUID"]

Provide a small JSON payload with `CI`, `Type` (`data` or `model`), `Services`, `Data_Rules`

#### Rule Payload (What each field means)

When you create a rule, you need to provide some details.  
Here’s what each field in the below example means:

- **CI**: The cyberinfrastructure (e.g., "OSC", "TACC) where this rule applies.  
- **Type**: The kind of rule (`data` or `model`).  
- **Active_From / Active_To**: When the rule should be active (start and optional end time).  
- **Services**: The list of services or steps this rule should trigger (e.g., `data-label`, `model-train`).  
- **Data_Rules**: Conditions for when the rule should trigger, like monitoring a folder.  
  - **Folder_Path**: The directory to watch.  
  - **Type**: The metric type (e.g., `count`).  
  - **Count**: The threshold (e.g., trigger when 10,000 new files appear).  
  - **Apps**: Which apps to trigger once the condition is met.    
- **Model_Rules**: Conditions for when the rule should trigger, like triggering if model accuracy is below certain threshold.  

Note: Modify these values to fit your project before creating the rule.

#### NOTE : The exact fields may change slightly as integration progresses and the project evolves.

In [62]:
# Example payload. Modify accordingly
rule_json = {
      "CI": "OSC",
      "Type": "data",
      "Active_From": "2025-12-09T12:00:00",
      "Active_To": None,
      "Services": [
        "data-label",
        "model-train"
      ],
      "Data_Rules": [
        {
          "Folder_Path": "/fs/ess/PAS1111/test/projectA/images",
          "Type": "count",
          "Count": 10000,
          "Apps": [
            "<TAPIS_APP_ID_1>",
            "<TAPIS_APP_ID_2>"
          ],
          "Sample_Images": True,
          "Wait_Manual": True
        }
      ],
      "Model_Rules": []
    }

In [63]:
rule_uuid = create_rule(rule_json)
print("Created:", rule_uuid)

Created: d93ee37c-2e72-4060-9554-8d4deea748c4


On success you’ll get back `rule_uuid`

### 6) List Rules

Fetches all your stored rules (scoped to your Tapis identity)

In [57]:
def list_rules():
    r = S.get(f"{BASE_URL}/rules", headers=_headers(), params=_params(), timeout=30)
    if r.status_code >= 400: 
        _raise_with_body(r)
    return r.json().get("items", [])

Displays the last 3 rules (if any) from your stored rules in a readable format.

In [64]:
pprint(list_rules()[-3:])

[{'Active_From': '2025-09-20T05:52:31.659870+00:00',
  'Active_To': None,
  'CI': 'OSC',
  'Data_Rules': [{'Apps': ['<TAPIS_APP_ID_1>', '<TAPIS_APP_ID_2>'],
                  'Count': 10000,
                  'Folder_Path': '/fs/ess/PAS1111/test/projectC/images',
                  'Sample_Images': True,
                  'Type': 'count',
                  'Wait_Manual': True}],
  'Rule_UUID': 'b1bdf421-3e66-4a6f-b51d-40e476602df6',
  'Services': ['data-label', 'model-train'],
  'TAPIS_UUID': 'molakalmuru@tacc',
  'Tapis_UserName': 'molakalmuru',
  'Type': 'data'},
 {'Active_From': '2025-09-20T05:52:56.415147+00:00',
  'Active_To': None,
  'CI': 'OSC',
  'Data_Rules': [{'Apps': ['<TAPIS_APP_ID_1>', '<TAPIS_APP_ID_2>'],
                  'Count': 10000,
                  'Folder_Path': '/fs/ess/PAS1111/test/projectB/images',
                  'Sample_Images': True,
                  'Type': 'count',
                  'Wait_Manual': True}],
  'Rule_UUID': '24947d07-97ea-4207-bebb-1be4ec47

### 7) Get a Rule (by UUID)

Retrieving a particular rule based on the `rule_uuid`

In [68]:
def get_rule(rule_uuid: str):
    r = S.get(f"{BASE_URL}/rules/{rule_uuid}", headers=_headers(), params=_params(), timeout=30)
    if r.status_code >= 400: 
        _raise_with_body(r)
    return r.json()

pprint(get_rule(rule_uuid))

{'Active_From': '2025-09-20T05:53:19.747211+00:00',
 'Active_To': None,
 'CI': 'TACC',
 'Data_Rules': [{'Apps': ['<TAPIS_APP_ID_1>', '<TAPIS_APP_ID_2>'],
                 'Count': 10000,
                 'Folder_Path': '/fs/ess/PAS1111/test/projectA/images',
                 'Sample_Images': True,
                 'Type': 'count',
                 'Wait_Manual': True}],
 'Rule_UUID': 'd93ee37c-2e72-4060-9554-8d4deea748c4',
 'Services': ['data-label', 'model-train'],
 'TAPIS_UUID': 'molakalmuru@tacc',
 'Tapis_UserName': 'molakalmuru',
 'Type': 'data'}


### 8) Update a Rule

- Send updates (e.g., set `Active_To` or `CI`)
- Confirm with a subsequent `get_rule` or `list_rules`

In [66]:
def update_rule(rule_uuid: str, updates: dict):
    r = S.patch(f"{BASE_URL}/rules/{rule_uuid}", headers=_headers(), params=_params(),
                json={"updates": updates}, timeout=30)
    if r.status_code >= 400: 
        _raise_with_body(r)
    return r.json()

In [67]:
updates = {
    "CI": "TACC",
}

result = update_rule(rule_uuid, updates)
print("Update Result:", result)

Update Result: {'ok': True}


## Add a updare call

### 9) Delete a Rule

- Delete by `rule_uuid`.
- Confirm with a subsequent `get_rule` or `list_rules`

In [69]:
def delete_rule(rule_uuid: str):
    r = S.delete(f"{BASE_URL}/rules/{rule_uuid}", headers=_headers(), params=_params(), timeout=30)
    if r.status_code >= 400: 
        _raise_with_body(r)
    return r.json()

pprint(delete_rule(rule_uuid))

{'ok': True}


### Notes

- The API validates your token **per request** (stateless).
- Rules store identity (`TAPIS_UUID`, `Tapis_UserName`), not raw tokens.
- For dev vs. prod: only the `BASE_URL` changes.

### Wrap-Up

You now have a minimal flow to **create, read, update, delete** rules against the UCA Rule Engine.  
Point `BASE_URL` to any environment and repeat the same steps.